В цій частині ми знову будемо працювати з датасетом fasion_mnist.

На відміну від попереднього завдання вам пропонується створити згорткову нейромережу, що використовує VGG16 в якості згорткової основи.

Навчіть отриману мережу на даних із датасету fasion_mnist. Спробуйте досягти максимально можливої точності класифікації за рахунок маніпуляції параметрами мережі. Під час навчання використовуйте прийоми донавчання та виділення ознак.

Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання. Зробіть висновки.

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy import ndimage
import numpy as np

In [7]:
# Завантажимо та підготуємо дані
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Конвертація чорно-білих зображень у зображення RGB
x_train_rgb = np.stack((x_train,) * 3, axis=-1)
x_test_rgb = np.stack((x_test,) * 3, axis=-1)

# Зміна розміру та нормалізація
x_train_rgb = tf.image.resize(x_train_rgb, (32, 32)) / 255.0
x_test_rgb = tf.image.resize(x_test_rgb, (32, 32)) / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

2024-01-17 22:37:05.626315: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 737280000 exceeds 10% of free system memory.
2024-01-17 22:37:06.797878: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 737280000 exceeds 10% of free system memory.


In [3]:
# Завантажимо VGG16 без верхнього повнозв'язного шару та ваг
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Додамо верхній повнозв'язний шар
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# Заморозимо ваги VGG16
base_model.trainable = False

# Скомпілюємо модель
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 (523.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [12]:
# Використовуємо прийоми донавчання та виділення ознак
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [15]:
# Навчання моделі
model.fit(datagen.flow(x_train_rgb.numpy().reshape((-1, 32, 32, 3)), y_train, batch_size=64),
          steps_per_epoch=len(x_train_rgb) / 64,
          epochs=10,
          validation_data=(x_test_rgb.numpy().reshape((-1, 32, 32, 3)), y_test))


# Оцінка точності на тестовому наборі
test_loss, test_acc = model.evaluate(x_test_rgb, y_test)
print("Точність на тестовому наборі:", test_acc)

/home/nomd/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1862: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/home/nomd/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1872: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/10
937/937 [==============================] - 329s 348ms/step - loss: 0.8318 - accuracy: 0.7026 - val_loss: 0.6470 - val_accuracy: 0.7674
Epoch 2/10
937/937 [==============================] - 326s 348ms/step - loss: 0.6624 - accuracy: 0.7592 - val_loss: 0.6168 - val_accuracy: 0.7677
Epoch 3/10
937/937 [==============================] - 330s 352ms/step - loss: 0.6234 - accuracy: 0.7721 - val_loss: 0.5682 - val_accuracy: 0.7895
Epoch 4/10
937/937 [==============================] - 321s 343ms/step - loss: 0.6035 - accuracy: 0.7762 - val_loss: 0.5337 - val_accuracy: 0.8077
Epoch 5/10
937/937 [==============================] - 325s 347ms/step - loss: 0.5880 - accuracy: 0.7819 - val_loss: 0.5378 - val_accuracy: 0.7944
Epoch 6/10
937/937 [==============================] - 329s 351ms/step - loss: 0.5778 - accuracy: 0.7855 - val_loss: 0.5307 - val_accuracy: 0.8035
Epoch 7/10
937/937 [==============================] - 331s 353ms/step - loss: 0.5733 - accuracy: 0.7886 - val_loss: 0.5036 -

2024-01-17 23:39:33.107606: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:761 : INVALID_ARGUMENT: convolution input must be 4-dimensional: [32,32,32,3,1]


InvalidArgumentError: Graph execution error:

Detected at node sequential/vgg16/block1_conv1/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/nomd/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/nomd/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/nomd/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/nomd/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/nomd/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", l